In [1]:
# Execute this cell and fully understand it
# [NOTE] Do not modify this cell
using JuMP
import DataFrames
using CPLEX
import Plots
import SparseArrays
using LinearAlgebra

# use this data structure; notation aligns with the problem description
mutable struct Data
    t::Matrix{Float64}
    l::Vector{Float64}
    u::Vector{Float64}
    d::Vector{Float64}
    s::Vector{Float64}
    q::Float64
    n::Int64

    # initializer
    function Data(xcoord::Vector{Float64}, ycoord::Vector{Float64}, demand::Vector{Float64}, l::Vector{Float64}, u::Vector{Float64}, s::Vector{Float64}, q::Float64)
        # number of customers
        n = length(xcoord)-1 
        # compute travel time
        t = [norm([xcoord[i]; ycoord[i]] - [xcoord[j]; ycoord[j]], 2) for i=1:n+1, j=1:n+1]
    
        d = demand
        new(t, l, u, d, s, q, n)
    end

end


In [2]:
# [NOTE] Do not modify this cell

# Below is the problem instance
A=[1	45.00	68.00	10.00	912.00	967.00	90.00
2	45.00	70.00	30.00	825.00	870.00	90.00
3	42.00	66.00	10.00	65.00	146.00	90.00
4	42.00	68.00	10.00	727.00	782.00	90.00
5	42.00	65.00	10.00	15.00	67.00	90.00
6	40.00	69.00	20.00	621.00	702.00	90.00
7	40.00	66.00	20.00	170.00	225.00	90.00
8	38.00	68.00	20.00	255.00	324.00	90.00
9	38.00	70.00	10.00	534.00	605.00	90.00
10	35.00	66.00	10.00	357.00	410.00	90.00
11	35.00	69.00	10.00	448.00	505.00	90.00
12	25.00	85.00	20.00	652.00	721.00	90.00
13	22.00	75.00	30.00	30.00	92.00	90.00
14	22.00	85.00	10.00	567.00	620.00	90.00
15	20.00	80.00	40.00	384.00	429.00	90.00
16	20.00	85.00	40.00	475.00	528.00	90.00
17	18.00	75.00	20.00	99.00	148.00	90.00
18	15.00	75.00	20.00	179.00	254.00	90.00
19	15.00	80.00	10.00	278.00	345.00	90.00
20	30.00	50.00	10.00	10.00	73.00	90.00
21	30.00	52.00	20.00	914.00	965.00	90.00
22	28.00	52.00	20.00	812.00	883.00	90.00
23	28.00	55.00	10.00	732.00	777.00	90.00
24	25.00	50.00	10.00	65.00	144.00	90.00
25	25.00	52.00	40.00	169.00	224.00	90.00
26	25.00	55.00	10.00	622.00	701.00	90.00
27	23.00	52.00	10.00	261.00	316.00	90.00
28	23.00	55.00	20.00	546.00	593.00	90.00
29	20.00	50.00	10.00	358.00	405.00	90.00
30	20.00	55.00	10.00	449.00	504.00	90.00
31	10.00	35.00	20.00	200.00	237.00	90.00
32	10.00	40.00	30.00	31.00	100.00	90.00
33	8.00	40.00	40.00	87.00	158.00	90.00
34	8.00	45.00	20.00	751.00	816.00	90.00
35	5.00	35.00	10.00	283.00	344.00	90.00
36	5.00	45.00	10.00	665.00	716.00	90.00
37	2.00	40.00	20.00	383.00	434.00	90.00
38	0.00	40.00	30.00	479.00	522.00	90.00
39	0.00	45.00	20.00	567.00	624.00	90.00
40	35.00	30.00	10.00	264.00	321.00	90.00
41	40.00	50.00	0.00	0.00	1236.00	0.00
]
# A[:,1] represents the customer indices where the last customer represents the depot
xcoord = A[:,2] # x-coordinate
ycoord = A[:,3] # y-coordinate
demand = A[:,4] # demand
l = A[:,5] # start time of service window
u = A[:,6] # ending time of service window
s = A[:,7] # service time
q = 200.0 # vehicle capacity

data = Data(xcoord, ycoord, demand, l, u, s, q)

Data([0.0 2.0 … 39.293765408777 18.681541692269406; 2.0 0.0 … 41.23105625617661 20.615528128088304; … ; 39.293765408777 41.23105625617661 … 0.0 20.615528128088304; 18.681541692269406 20.615528128088304 … 20.615528128088304 0.0], [912.0, 825.0, 65.0, 727.0, 15.0, 621.0, 170.0, 255.0, 534.0, 357.0  …  31.0, 87.0, 751.0, 283.0, 665.0, 383.0, 479.0, 567.0, 264.0, 0.0], [967.0, 870.0, 146.0, 782.0, 67.0, 702.0, 225.0, 324.0, 605.0, 410.0  …  100.0, 158.0, 816.0, 344.0, 716.0, 434.0, 522.0, 624.0, 321.0, 1236.0], [10.0, 30.0, 10.0, 10.0, 10.0, 20.0, 20.0, 20.0, 10.0, 10.0  …  30.0, 40.0, 20.0, 10.0, 10.0, 20.0, 30.0, 20.0, 10.0, 0.0], [90.0, 90.0, 90.0, 90.0, 90.0, 90.0, 90.0, 90.0, 90.0, 90.0  …  90.0, 90.0, 90.0, 90.0, 90.0, 90.0, 90.0, 90.0, 90.0, 0.0], 200.0, 40)

## 1. Implement the MILP formulation by completing the statements below:


In [3]:
mutable struct NaiveMIP
    model::Model
    M::Int64
    x::Array{VariableRef, 3}
    y::Matrix{VariableRef}

    # initializer
    function NaiveMIP(data::Data)
        n = data.n
        t = data.t
        q = data.q
        d = data.d
        u = data.u
        l = data.l
        s = data.s
        M = n
        model = Model(CPLEX.Optimizer)
        @variable(model, x[1:n+1,1:n+1,1:M], Bin)
        @variable(model, y[1:n+1,1:M])

        # Constraint: Each customer must be serviced exactly once
        for i in 1:n
            @constraint(model, sum(x[i,j,k] for j in 1:n+1, k in 1:M) == 1)
        end

        # Constraint: Each route originates and ends at vertex 0
        for k in 1:M
            @constraint(model, sum(x[1,i,k] for i in 2:n+1) == 1)
            @constraint(model, sum(x[i,1,k] for i in 2:n+1) == 1)
        end

        # Constraint: Time Window Constraints are satisfied
        for i in 1:n, j in 1:n, k in 1:M
            Mij = max(u[i] + s[i] + t[i,j] - l[j], 0)
            @constraint(model, y[i,k] - y[j,k] + Mij * (1 - x[i,j,k]) >= s[i])
        end

        # Constraint: Capacity constraints of the vehicles are fulfilled
        for k in 1:M
            @constraint(model, sum(d[i] * x[i,j,k] for i in 1:n, j in 1:n+1) <= q)
        end

        # Objective: Minimize total number of vehicles utilized
        @objective(model, Min, sum(x[i,j,k] for i in 1:n+1, j in 1:n+1, k in 1:M))

        return new(model, M, x, y)
    end
end




### solve the MILP and report the result in the PDF file.

In [4]:
# Solve the MILP
nmip = NaiveMIP(data)
optimize!(nmip.model)
solution_summary(nmip.model)

Version identifier: 22.1.1.0 | 2022-11-27 | 9160aff4d
Infeasibility row 'c1':  0  = -39.
Presolve time = 0.16 sec. (93.70 ticks)


* Solver : CPLEX

* Status
  Result count       : 0
  Termination status : INFEASIBLE
  Message from the solver:
  "integer infeasible"

* Candidate solution (result #1)
  Primal status      : NO_SOLUTION
  Dual status        : NO_SOLUTION
  Objective bound    : 0.00000e+00

* Work counters
  Solve time (sec)   : 2.52000e-01
  Simplex iterations : 0
  Barrier iterations : 0
  Node count         : 0


## 2. Implement Column Generation
### (a). Implement the master problem by completing the statements below:

In [5]:
mutable struct Master
    model::Model
    z::Vector{VariableRef} # variables
    demand_constr::Vector{<:ConstraintRef} # constraints
    routes::Vector{Vector{Float64}}
    P::Int64 # number of routes
end

function Master(data)
    n = data.n

    # statement 1: construct initial naive configurations
    routes = Vector{Vector{Float64}}()
    for i in 1:n
        route = zeros(Int,n)
        route[i] = 1
        push!(routes, route)
    end

    # statement 2: Let P denote the number of initial routes
    P = length(routes)

    # define empty model
    model = Model(CPLEX.Optimizer)

    # mute the output
    set_silent(model)

    # statement 3: add variable z, objective function, and constraint named 'demand_constr' to model
    @variable(model, z[1:n] >= 0)
    @objective(model, Min, sum(z))
    @constraint(model, demand_constr[i=1:P], routes[i]'*z==1)

    return Master(model, z, demand_constr, routes, P)
end

Master

### (b). Implement the pricing problem by completing the statements below:

In [6]:
mutable struct Sub
    model::Model
    a::Vector{VariableRef} # variables
end

function Sub(data)
    l = data.l
    u = data.u
    t = data.t
    d = data.d
    n = data.n

    # initial dummy dual values
    π̂ = zeros(Float64, n)

    model = Model(CPLEX.Optimizer)
    @variable(model, x[1:n+1, 1:n+1], Bin)
    @variable(model, y[1:n+1])
    @variable(model, a[1:n])

    set_silent(model)

    # statement 1: add constraints and objective to model
    @constraint(model, [i=1:n, j=1:n+1; i != j], y[j] >= y[i] + t[i, j] + s[i] - M * (1 - x[i, j]))
    @constraint(model, [i=1:n], l[i] <= y[i] <= u[i])
    @constraint(model, [i=1:n], sum(x[i, j] for j=1:n+1) == sum(x[j, i] for j=1:n+1))
    @constraint(model, sum(x[1, j] for j=2:n+1) == 1)
    @constraint(model, sum(x[j, 1] for j=2:n+1) == 1)
    @constraint(model, [i=1:n], a[i] == sum(x[i, j] for j=1:n+1))
    @objective(model, Max, 1 - sum(π̂[i] * a[i] for i=1:n))

    return Sub(model, a)
end

Sub

### (c) Implement `runCG(master::Master, sub::Sub, data::Data)` by completing the following statements:

In [7]:
function runCG(master::Master, sub::Sub, data::Data)
    while true
        # solve the restricted master problem
        optimize!(master.model)
        println(value.(master.z))
        println(objective_value(master.model))
        # get duals
        π = dual.(master.demand_constr)

        # update the subproblem objective function using the dual info
        set_objective_coefficient.(sub.model, sub.a, -π)
        # solve the subproblem
        optimize!(sub.model)
        # obtain the optimal value of the sub, which corresponds to the reduced cost of the column generated by the sub
        reduced_cost = objective_value(sub.model)
        if reduced_cost < -1e-8
            # add the column with negative reduced cost
            â = value.(sub.a)
            push!(master.routes, â)
            push!(master.z, @variable(master.model, lower_bound = 0))
            set_objective_coefficient(master.model, master.z[end], 1.0)
            set_normalized_coefficient.(master.demand_constr, master.z[end], â)
            println("Found new pattern. Total patterns = $(length(master.routes))")
        else
            break
        end
    end
end


runCG (generic function with 1 method)

### run the CG and report the result in the PDF file.

In [8]:
# generate the master problem for CG
master = Master(data)
sub = Sub(data)

tic = time()
runCG(master, sub, data)
toc = time()

solution_time = toc-tic
println("solution time: ", solution_time)

objective_value(master.model)


LoadError: UndefVarError: `M` not defined